Дан граф текста, множество вершин совпадает со словарем текста, связи являются взвешенными и показывают как часто два слова находились в одном окне длины N. Найти слова с наибольшей мерой посредничества.



In [40]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk import everygrams
nltk.download('punkt')
import re
import collections
import itertools
import networkx as nx
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!pip install pymorphy2

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 8.2MB 3.9MB/s 


In [3]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Читаем какой-нибудь текст, например, Джека Лондона

In [4]:
def read_file (filename):
    with open (filename, encoding = 'utf-8') as file:
        text = file.read()
    return text 

In [5]:
filename = 'D_London.txt'

In [7]:
text = read_file (filename)

In [8]:
text

'Джек Лондон. Зов предков\n\n\n---------------------------------------------------------------\n Изд. "Правда", 1984 г.\n OCR Палек, 1998 г.\n---------------------------------------------------------------\n\n\n\nI. К ПЕРВОБЫТНОЙ ЖИЗНИ\n\n\n     Древние бродячие инстинкты\n     Перетирают цепь привычки и веков,\n     И, просыпаясь от глубокой спячки,\n     Вновь дикий зверь выходит из оков\n\n     Бэк не читал газет и потому не знал,  что  надвигается  беда -- и не на\nнего одного, а на всех собак с сильными  мышцами и длинной,  теплой шерстью,\nсколько их ни  было от залива  Пюджет до Сан-Диего.  И все оттого, что люди,\nощупью пробираясь сквозь полярный мрак, нашли желтый металл, а  пароходные и\nтранспортные  компании раструбили повсюду об этой находке, -- и тысячи людей\nринулись  на Север.  Этим людям нужны были  собаки крупной  породы, сильные,\nгодные для тяжелой работы, с густой и длинной шерстью, которая защитит их от\nморозов.\n     Бэк жил в большом доме, в солнечной долине 

Возьмем для примера окна длины 4

In [9]:
words = []
n = 4

Берем из него только первую главу, чтобы побыстрее работало

In [10]:
def part_of_text (text):
    chapter_1 = []
    chapters = re.split(r'I+\.', text)
    chapter_1 = chapters[1]
    return chapter_1

In [11]:
chapter_1 = part_of_text (text)

Преобразуем его для удобства

In [12]:
chapter_1 = chapter_1.lower()

In [13]:
chapter_2 = re.sub('[^\w \n]+', '', chapter_1)
chapter = re.sub('\s+', ' ', chapter_2)
chapter

' к первобытной жизни древние бродячие инстинкты перетирают цепь привычки и веков и просыпаясь от глубокой спячки вновь дикий зверь выходит из оков бэк не читал газет и потому не знал что надвигается беда и не на него одного а на всех собак с сильными мышцами и длинной теплой шерстью сколько их ни было от залива пюджет до сандиего и все оттого что люди ощупью пробираясь сквозь полярный мрак нашли желтый металл а пароходные и транспортные компании раструбили повсюду об этой находке и тысячи людей ринулись на север этим людям нужны были собаки крупной породы сильные годные для тяжелой работы с густой и длинной шерстью которая защитит их от морозов бэк жил в большом доме в солнечной долине сантаклара место это люди называли усадьбой судьи миллера дом стоял в стороне от дороги полускрытый за деревьями и сквозь ветви виднелась только веранда просторная и тенистая окружавшая дом со всех сторон к дому вели посыпанные гравием дорожки они вились по широким лужайкам под стройными тополями ветви 

Токенизируем текст

In [14]:
tokenized_text = nltk.word_tokenize(chapter)

Превратим список токенов в список лемм и создадим n-граммы

In [15]:
lemmas_list = []

In [16]:
for word in tokenized_text:
  ana = morph.parse(word)[0].normal_form
  lemmas_list.append(ana)

In [17]:
def n_grams_list (tokenized_text, n):
    n_grams = list(ngrams(tokenized_text, n))
    return n_grams

Список кортежей окон длины n

In [18]:
n_grams = n_grams_list (lemmas_list, n)
print (n_grams)

[('к', 'первобытный', 'жизнь', 'древний'), ('первобытный', 'жизнь', 'древний', 'бродячий'), ('жизнь', 'древний', 'бродячий', 'инстинкт'), ('древний', 'бродячий', 'инстинкт', 'перетирать'), ('бродячий', 'инстинкт', 'перетирать', 'цепь'), ('инстинкт', 'перетирать', 'цепь', 'привычка'), ('перетирать', 'цепь', 'привычка', 'и'), ('цепь', 'привычка', 'и', 'век'), ('привычка', 'и', 'век', 'и'), ('и', 'век', 'и', 'просыпаться'), ('век', 'и', 'просыпаться', 'от'), ('и', 'просыпаться', 'от', 'глубокий'), ('просыпаться', 'от', 'глубокий', 'спячка'), ('от', 'глубокий', 'спячка', 'вновь'), ('глубокий', 'спячка', 'вновь', 'дикий'), ('спячка', 'вновь', 'дикий', 'зверь'), ('вновь', 'дикий', 'зверь', 'выходить'), ('дикий', 'зверь', 'выходить', 'из'), ('зверь', 'выходить', 'из', 'окова'), ('выходить', 'из', 'окова', 'бэк'), ('из', 'окова', 'бэк', 'не'), ('окова', 'бэк', 'не', 'читать'), ('бэк', 'не', 'читать', 'газета'), ('не', 'читать', 'газета', 'и'), ('читать', 'газета', 'и', 'потому'), ('газета', 'и

Необходимые для дальнейших действий пустые списки

In [19]:
list_of_pairs = []
pairs_list = []
word_pairs = []

Делам список всевозможных комбинаций двух слов по каждому из окон 

In [20]:
for n_gram in n_grams:
  list_of_pairs.extend(list(itertools.combinations(n_gram, 2))) 

In [21]:
list_of_pairs

[('к', 'первобытный'),
 ('к', 'жизнь'),
 ('к', 'древний'),
 ('первобытный', 'жизнь'),
 ('первобытный', 'древний'),
 ('жизнь', 'древний'),
 ('первобытный', 'жизнь'),
 ('первобытный', 'древний'),
 ('первобытный', 'бродячий'),
 ('жизнь', 'древний'),
 ('жизнь', 'бродячий'),
 ('древний', 'бродячий'),
 ('жизнь', 'древний'),
 ('жизнь', 'бродячий'),
 ('жизнь', 'инстинкт'),
 ('древний', 'бродячий'),
 ('древний', 'инстинкт'),
 ('бродячий', 'инстинкт'),
 ('древний', 'бродячий'),
 ('древний', 'инстинкт'),
 ('древний', 'перетирать'),
 ('бродячий', 'инстинкт'),
 ('бродячий', 'перетирать'),
 ('инстинкт', 'перетирать'),
 ('бродячий', 'инстинкт'),
 ('бродячий', 'перетирать'),
 ('бродячий', 'цепь'),
 ('инстинкт', 'перетирать'),
 ('инстинкт', 'цепь'),
 ('перетирать', 'цепь'),
 ('инстинкт', 'перетирать'),
 ('инстинкт', 'цепь'),
 ('инстинкт', 'привычка'),
 ('перетирать', 'цепь'),
 ('перетирать', 'привычка'),
 ('цепь', 'привычка'),
 ('перетирать', 'цепь'),
 ('перетирать', 'привычка'),
 ('перетирать', 'и'),


Проблема в том, что в тех комбинациях есть варианты, например ('на', 'я') и ('я', 'на'), но по сути это одно и то же ребро, так как наш граф неориентированный. Превратим кортежи в списки и отсортируем каждый из них, тогда "названия" одного и того же ребра будет одинаково.

In [22]:
for element in list_of_pairs:
  pairs_list.append(list(element))

In [23]:
number1 = 0
for pair in pairs_list:
  number1 += 1
  pair.sort()

Это число отражает сумму количеств вхождений каждого из ребер

In [24]:
number1

19758

Далее мы переделаем опять списки в кортежи, так как списки не могут быть ключом словаря

In [25]:
for pair in pairs_list:
  word_pairs.append(tuple(pair))

Создадим частотный словарь

In [26]:
counter = collections.Counter(word_pairs)

In [27]:
counter

Counter({('к', 'первобытный'): 1,
         ('жизнь', 'к'): 1,
         ('древний', 'к'): 1,
         ('жизнь', 'первобытный'): 2,
         ('древний', 'первобытный'): 2,
         ('древний', 'жизнь'): 3,
         ('бродячий', 'первобытный'): 1,
         ('бродячий', 'жизнь'): 2,
         ('бродячий', 'древний'): 3,
         ('жизнь', 'инстинкт'): 1,
         ('древний', 'инстинкт'): 2,
         ('бродячий', 'инстинкт'): 3,
         ('древний', 'перетирать'): 1,
         ('бродячий', 'перетирать'): 2,
         ('инстинкт', 'перетирать'): 3,
         ('бродячий', 'цепь'): 1,
         ('инстинкт', 'цепь'): 2,
         ('перетирать', 'цепь'): 3,
         ('инстинкт', 'привычка'): 1,
         ('перетирать', 'привычка'): 2,
         ('привычка', 'цепь'): 3,
         ('и', 'перетирать'): 1,
         ('и', 'цепь'): 2,
         ('и', 'привычка'): 4,
         ('век', 'цепь'): 1,
         ('век', 'привычка'): 2,
         ('век', 'и'): 6,
         ('и', 'и'): 15,
         ('и', 'просыпаться'): 4,


Посмотрим, какое количество ребер получилось в нашем графе

In [28]:
num = 0

In [29]:
for key in counter:
  num += 1

In [30]:
num

7964

Далее приведем наш словарь к иному виду: сделаем список списков, в каждом из которых будет три элемента (две смежные вершины и число, которое отражает как часто эти два слова находились в одном окне длины n)

In [44]:
data = []

In [45]:
for pair, number in counter.items():
  edge = []
  for word in pair:
    edge.append(word)
  freq_dict = {}
  freq_dict['freq'] = number
  edge.append(freq_dict)
  data.append(edge)


In [46]:
data

[['к', 'первобытный', {'freq': 1}],
 ['жизнь', 'к', {'freq': 1}],
 ['древний', 'к', {'freq': 1}],
 ['жизнь', 'первобытный', {'freq': 2}],
 ['древний', 'первобытный', {'freq': 2}],
 ['древний', 'жизнь', {'freq': 3}],
 ['бродячий', 'первобытный', {'freq': 1}],
 ['бродячий', 'жизнь', {'freq': 2}],
 ['бродячий', 'древний', {'freq': 3}],
 ['жизнь', 'инстинкт', {'freq': 1}],
 ['древний', 'инстинкт', {'freq': 2}],
 ['бродячий', 'инстинкт', {'freq': 3}],
 ['древний', 'перетирать', {'freq': 1}],
 ['бродячий', 'перетирать', {'freq': 2}],
 ['инстинкт', 'перетирать', {'freq': 3}],
 ['бродячий', 'цепь', {'freq': 1}],
 ['инстинкт', 'цепь', {'freq': 2}],
 ['перетирать', 'цепь', {'freq': 3}],
 ['инстинкт', 'привычка', {'freq': 1}],
 ['перетирать', 'привычка', {'freq': 2}],
 ['привычка', 'цепь', {'freq': 3}],
 ['и', 'перетирать', {'freq': 1}],
 ['и', 'цепь', {'freq': 2}],
 ['и', 'привычка', {'freq': 4}],
 ['век', 'цепь', {'freq': 1}],
 ['век', 'привычка', {'freq': 2}],
 ['век', 'и', {'freq': 6}],
 ['и'

Но находить слова с наибольшей степень посредничества все-таки проще через частотный словарь, на мой взгляд.
Выведем топ-10 пар слов

Что-то про степень посредничества и про сам граф 

In [47]:
G = nx.Graph()

In [48]:
G.add_edges_from(data)

In [49]:
G.edges(data=True)

EdgeDataView([('к', 'первобытный', {'freq': 1}), ('к', 'жизнь', {'freq': 1}), ('к', 'древний', {'freq': 1}), ('к', 'с', {'freq': 1}), ('к', 'весь', {'freq': 4}), ('к', 'сторона', {'freq': 3}), ('к', 'дом', {'freq': 3}), ('к', 'вели', {'freq': 2}), ('к', 'посыпать', {'freq': 1}), ('к', 'фунт', {'freq': 1}), ('к', 'вес', {'freq': 2}), ('к', 'если', {'freq': 3}), ('к', 'они', {'freq': 3}), ('к', 'ещё', {'freq': 3}), ('к', 'прибавить', {'freq': 1}), ('к', 'большой', {'freq': 1}), ('к', 'порка', {'freq': 2}), ('к', 'страсть', {'freq': 3}), ('к', 'китайский', {'freq': 5}), ('к', 'лотерея', {'freq': 5}), ('к', 'к', {'freq': 1}), ('к', 'тот', {'freq': 3}), ('к', 'же', {'freq': 2}), ('к', 'у', {'freq': 1}), ('к', 'быть', {'freq': 2}), ('к', 'равносильно', {'freq': 2}), ('к', 'приказание', {'freq': 3}), ('к', 'он', {'freq': 8}), ('к', 'удивление', {'freq': 2}), ('к', 'верёвка', {'freq': 1}), ('к', 'когда', {'freq': 1}), ('к', 'кидаться', {'freq': 3}), ('к', 'решётка', {'freq': 6}), ('к', 'дрожат

In [56]:
dict_BC = nx.betweenness_centrality(G)

float

Выведем топ-10 слов с наибольшей степенью посредничества 

In [58]:
for key in sorted(dict_BC, key=dict_BC.get, reverse = True)[:10]:
  print(key, dict_BC[key])

и 0.26718394997923767
он 0.22603129894542126
в 0.11908759431281728
бэк 0.06983206895193828
на 0.06577147685006703
с 0.05081064532242502
быть 0.04472493378546806
не 0.04376583392682087
они 0.038775212638445584
а 0.03451938941365272
